In [1]:
from utils import *
%matplotlib inline

# Use GPUs

<center><img src="support/gpu.gif" width=600></center>

We often use GPUs to train and deploy neural networks, because it offers significant more computation power compared to CPUs. In this tutorial we will introduce how to use GPUs with MXNet.

First, make sure you have at least one Nvidia GPU in your machine and CUDA
properly installed. Other GPUs such as AMD and Intel GPUs are not supported
yet. Then be sure you have
[installed the GPU-enabled version of MXNet](mxnet_packages.md).

In [2]:
# If you pip installed the plain `mxnet` before, uncomment the
# following two lines to install the GPU version. You may need to
# replace `cu91` according to your CUDA version.
#
# !pip uninstall mxnet
# !pip install mxnet-cu91

In [3]:
from mxnet import nd, gpu, gluon, autograd

from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms

from time import time

## Allocate data to a GPU

You may notice that MXNet's NDArray is very similar to Numpy. One major difference is NDArray has a `context` attribute that specifies which device this array is on. By default, it is `cpu()`. Now we will change it to the first GPU. You can use `gpu()` or `gpu(0)` to indicate the first GPU.

In [4]:
x = nd.ones((3,4), ctx=gpu())
x


[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @gpu(0)>

For a CPU, MXNet will allocate data on main memory, and try to use all CPU cores as possible, even if there is more than one CPU socket. While if there are multiple GPUs, MXNet needs to specify which GPUs the NDArray will be allocated.

If there is at least one more GPU. We can create another NDArray and assign it there. (Note: If you only have one GPU, then you will see an error when you run the below). Here we copy `x` to the second GPU, `gpu(1)`:

In [5]:
x.copyto(gpu(0))


[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @gpu(0)>

Several operators such as `print`, `asnumpy` and `asscalar`, will implicitly move data to main memory.


MXNet needs users to explicitly move data between devices. But several operators such as `print`, `asnumpy` and `asscalar`, will implicitly move data to main memory.

## Run an operation on a GPU

To perform an operation on a particular GPU, we only need to guarantee that the inputs of this operation are already on that GPU. The output will be allocated on the same GPU as well. Almost all operators in the `nd` module support running on a GPU.

In [6]:
y = nd.random.uniform(shape=(3,4), ctx=gpu(0))
x + y


[[ 1.66865087  1.17409194  1.38500249  1.24678314]
 [ 1.35134339  1.84042978  1.63699174  1.12846994]
 [ 1.17249882  1.93682063  1.59183455  1.94970965]]
<NDArray 3x4 @gpu(0)>

Remember that if the inputs are not on the same GPU, you will see an error.

## Run a neural network on a GPU

Similarly, to run a neural network on a GPU, we only need to copy/move the input data and parameters to the GPU. Let's reuse the previously defined LeNet.

In [7]:
net = nn.HybridSequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Flatten(),
        nn.Dense(120, activation="relu"),
        nn.Dense(84, activation="relu"),
        nn.Dense(10)
    )

And then load the saved parameters into GPU 0 directly, or use `net.collect_params().reset_ctx` to change the device.

### Load parameters

In [8]:
net.load_parameters('net.params', ctx=gpu(0))

input data on GPU 0. forward pass will run on GPU 0.

In [9]:
x = nd.random.uniform(shape=(1,1,28,28), ctx=gpu(0))

net(x)


[[ 0.93490076  0.32338083  0.53775805  0.14257076 -0.06618062 -1.16491938
   1.35422647 -1.33665538  0.36462927 -1.58663666]]
<NDArray 1x10 @gpu(0)>

## Multi-GPU training

Finally, we show how to use multiple GPUs to jointly train a neural network through data parallelism. Let's assume there are *n* GPUs. We split each data batch into *n* parts, and then each GPU will run the forward and backward passes using one part of the data.

Let's first copy the data definitions and the transform function from the [previous tutorial](predict.md).

In [10]:
batch_size = 512

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.13, 0.31)])

train_data = gluon.data.DataLoader(
    datasets.FashionMNIST(train=True).transform_first(transformer),
    batch_size, shuffle=True, num_workers=6)

The training loop is quite similar to what we introduced before. The major differences are highlighted in the following code.

In [11]:
# Diff 1: Use two GPUs for training.
devices = [gpu(0), gpu(1)]

# Diff 2: reinitialize the parameters and place them on multiple GPUs
net.initialize(force_reinit=True, ctx=devices)

# Loss and trainer are the same as before, lr scaled
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.1*len(devices)})

MXNetError: [17:51:10] src/engine/threaded_engine.cc:322: Check failed: exec_ctx.dev_id < device_count_ (1 vs. 1) Invalid GPU Id: 1, Valid device id should be less than device_count: 1

Stack trace returned 10 entries:
[bt] (0) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x36161a) [0x7f04adffd61a]
[bt] (1) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x361c31) [0x7f04adffdc31]
[bt] (2) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2a4c2bc) [0x7f04b06e82bc]
[bt] (3) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2c50a8c) [0x7f04b08eca8c]
[bt] (4) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2addca1) [0x7f04b0779ca1]
[bt] (5) /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2ae953a) [0x7f04b078553a]
[bt] (6) /home/ec2-user/anaconda3/envs/

In [ ]:
for epoch in range(10):
    train_loss = 0
    tic = time()
    for data, label in train_data:
        # Diff 3: split batch and load into corresponding devices
        data_list = gluon.utils.split_and_load(data, devices)
        label_list = gluon.utils.split_and_load(label, devices)
        # Diff 4: run forward and backward on each devices.
        with autograd.record():
            losses = [softmax_cross_entropy(net(X), y)
                      for X, y in zip(data_list, label_list)]
        for l in losses:
            l.backward()
        trainer.step(batch_size)
        # Diff 5: sum losses across each devices
        for i, l in enumerate(losses):
            train_loss += l.mean().asscalar()
            
    print("Epoch %d: Loss: %.3f, Perf %.1f img/sec" % (
        epoch, train_loss/len(train_data), 
        len(train_data)*batch_size/(time()-tic)))

<center><img src="support/fast.gif" width=800></center>